In [7]:
!pip -q install -U evaluate sacrebleu rouge-score chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently t

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### Setup (defines questions, refs, top_k, retrieve(), generate_text())

In [8]:
import os, json, random, numpy as np, torch, chromadb
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

BASE_DIR   = "/content/drive/MyDrive/agentic-rag-telecom-thesis"
TRACE_IN   = f"{BASE_DIR}/results/traces/baseline_traces.jsonl"
CHROMA_DIR = f"{BASE_DIR}/results/chroma_baseline"

assert os.path.exists(TRACE_IN), f"Missing: {TRACE_IN}"
assert os.path.exists(CHROMA_DIR), f"Missing: {CHROMA_DIR}"

# Load SAME questions used in baseline (first N_QA)
N_QA = 50
rows = [json.loads(l) for l in open(TRACE_IN, "r", encoding="utf-8")][:N_QA]
questions = [r["query"] for r in rows]
refs      = [r["reference"] for r in rows]
top_k     = rows[0].get("top_k", 5)

# Open SAME Chroma + embedder
client = chromadb.PersistentClient(path=CHROMA_DIR)
col = client.get_collection("doc2dial_baseline")
emb = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve(query, k=5):
    q_emb = emb.encode([query], normalize_embeddings=True).tolist()[0]
    res = col.query(query_embeddings=[q_emb], n_results=k, include=["documents","metadatas"])
    return res["documents"][0], res["metadatas"][0]

# FLAN-T5 generator (no transformers pipeline)
device = "cuda" if torch.cuda.is_available() else "cpu"
tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to(device)
model.eval()

@torch.inference_mode()
def generate_text(prompt: str, max_new_tokens: int = 256) -> str:
    inputs = tok(prompt, return_tensors="pt", truncation=True).to(device)
    out_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, num_beams=1)
    return tok.decode(out_ids[0], skip_special_tokens=True).strip()

print("Loaded QA:", len(questions), "| top_k:", top_k, "| device:", device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Loaded QA: 50 | top_k: 5 | device: cpu


### Installing + metrics objects (evaluate)

In [9]:
!pip -q install -U evaluate sacrebleu rouge-score

import evaluate
rouge = evaluate.load("rouge")
bleu  = evaluate.load("sacrebleu")

### Planner–Executor run (saves traces + summary)

In [10]:
import re, time, csv, os, json
import numpy as np

TRACE_OUT = f"{BASE_DIR}/results/traces/planner_traces.jsonl"
OUT_CSV   = f"{BASE_DIR}/results/planner_summary.csv"

PLANNER_SYS = """You are a planner for telecom support.
Given the QUESTION, write a short plan of 2-3 steps.
Return ONLY numbered steps, one per line (e.g., "1. ...")."""

EXECUTOR_SYS = """You are a telecom customer-support assistant.
Use ONLY the CONTEXT to answer. If not in context, say: I don't know."""

def make_plan(question: str) -> list[str]:
    plan_txt = generate_text(f"{PLANNER_SYS}\n\nQUESTION: {question}\nPLAN:")
    steps = []
    for line in plan_txt.splitlines():
        m = re.match(r"^\s*\d+\.\s*(.+)$", line.strip())
        if m:
            steps.append(m.group(1).strip())
    return steps[:3] if steps else [question]

def planner_executor_answer(question, k=5):
    t0 = time.time()
    steps = make_plan(question)
    action_queries = []

    all_docs, all_metas = [], []
    for s in steps:
        action_queries.append(s)
        docs, metas = retrieve(s, k=k)
        all_docs.extend(docs)
        all_metas.extend(metas)

    # de-dupe while keeping order
    seen, dedup_docs, dedup_metas = set(), [], []
    for d, m in zip(all_docs, all_metas):
        key = (m.get("doc_id"), m.get("chunk_id"))
        if key not in seen:
            seen.add(key)
            dedup_docs.append(d)
            dedup_metas.append(m)

    context = "\n\n".join([f"[{m.get('doc_id')}#{m.get('chunk_id')}] {d}" for d, m in zip(dedup_docs, dedup_metas)])
    prompt = f"{EXECUTOR_SYS}\n\nCONTEXT:\n{context}\n\nQUESTION: {question}\nANSWER:"
    ans = generate_text(prompt)
    return ans, time.time()-t0, steps, action_queries, dedup_docs, dedup_metas

preds, lats = [], []
os.makedirs(os.path.dirname(TRACE_OUT), exist_ok=True)

with open(TRACE_OUT, "w", encoding="utf-8") as f:
    for i, (q, ref) in enumerate(zip(questions, refs), 1):
        ans, dt, steps, aq, ctxs, metas = planner_executor_answer(q, k=top_k)
        preds.append(ans); lats.append(dt)
        f.write(json.dumps({
            "i": i, "query": q, "prediction": ans, "reference": ref,
            "latency_s": float(dt), "top_k": int(top_k),
            "plan_steps": steps, "action_queries": aq,
            "contexts": ctxs[:top_k*3],
            "retrieved": [{"doc_id": m.get("doc_id"), "chunk_id": m.get("chunk_id")} for m in metas]
        }, ensure_ascii=False) + "\n")

rou = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
ble = bleu.compute(predictions=preds, references=[[r] for r in refs])
lat_mean, lat_med, lat_p95 = float(np.mean(lats)), float(np.median(lats)), float(np.percentile(lats, 95))

print("=== Planner–Executor ===")
print("ROUGE-L:", rou["rougeL"])
print("BLEU:", ble["score"])
print("Latency mean/median/p95:", lat_mean, lat_med, lat_p95)
print("Saved traces:", TRACE_OUT)

with open(OUT_CSV, "w", newline="", encoding="utf-8") as fp:
    w = csv.DictWriter(fp, fieldnames=["system","rougeL","bleu","lat_mean_s","lat_median_s","lat_p95_s","n_pairs","top_k"])
    w.writeheader()
    w.writerow({
        "system":"Planner–Executor (Doc2Dial)",
        "rougeL":rou["rougeL"], "bleu":ble["score"],
        "lat_mean_s":lat_mean, "lat_median_s":lat_med, "lat_p95_s":lat_p95,
        "n_pairs":len(questions), "top_k":top_k
    })
print("Saved summary:", OUT_CSV)

=== Planner–Executor ===
ROUGE-L: 0.04487328628913493
BLEU: 0.35366353568338826
Latency mean/median/p95: 9.284378247261047 6.655436992645264 22.116019797325134
Saved traces: /content/drive/MyDrive/agentic-rag-telecom-thesis/results/traces/planner_traces.jsonl
Saved summary: /content/drive/MyDrive/agentic-rag-telecom-thesis/results/planner_summary.csv


In [1]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.5/466.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: jiter
    Found existing ins

Tracking planner Trace

In [9]:
from google.colab import drive
drive.mount("/content/drive")

!ls -lah /content/drive/MyDrive/agentic-rag-telecom-thesis/results/traces | sed -n '1,200p'
!find /content/drive/MyDrive/agentic-rag-telecom-thesis -maxdepth 6 -type f -name "planner_traces.jsonl" -print

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 603K
-rw------- 1 root root  45K Feb 27 09:33 baseline_traces.jsonl
-rw------- 1 root root 289K Feb 27 11:50 planner_traces.jsonl
-rw------- 1 root root 270K Feb 27 11:13 react_traces.jsonl
/content/drive/MyDrive/agentic-rag-telecom-thesis/results/traces/planner_traces.jsonl


### RAGAS for Planner–Executor (N=20)

In [12]:
import os, json
from datasets import Dataset
from ragas import evaluate as ragas_evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision  # legacy metric OBJECTS (works)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

assert os.environ.get("OPENAI_API_KEY"), "Set OPENAI_API_KEY first."

TRACE_PLANNER = "/content/drive/MyDrive/agentic-rag-telecom-thesis/results/traces/planner_traces.jsonl"
BASE_DIR = "/content/drive/MyDrive/agentic-rag-telecom-thesis"

N = 20
rows = [json.loads(l) for l in open(TRACE_PLANNER, "r", encoding="utf-8")][:N]

ds = Dataset.from_dict({
    "question": [r["query"] for r in rows],
    "answer": [r["prediction"] for r in rows],
    "contexts": [r.get("contexts", []) for r in rows],
    "ground_truth": [r["reference"] for r in rows],
})

llm_eval = ChatOpenAI(model="gpt-4o-mini", temperature=0)
emb_eval = OpenAIEmbeddings(model="text-embedding-3-small")

res = ragas_evaluate(
    ds,
    metrics=[faithfulness, answer_relevancy, context_precision],
    llm=llm_eval,
    embeddings=emb_eval
)

df = res.to_pandas()
means = df[["faithfulness","answer_relevancy","context_precision"]].mean().to_dict()
print("Planner RAGAS means on N=", N, means)

out_csv = f"{BASE_DIR}/results/planner_ragas_fixed.csv"
df.to_csv(out_csv, index=False)
print("Saved:", out_csv)

/tmp/ipython-input-677/3468083264.py:4: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_precision  # legacy metric OBJECTS (works)
/tmp/ipython-input-677/3468083264.py:4: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy, context_precision  # legacy metric OBJECTS (works)
/tmp/ipython-input-677/3468083264.py:4: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_preci

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Planner RAGAS means on N= 20 {'faithfulness': 0.4956140350877193, 'answer_relevancy': 0.17485477705665056, 'context_precision': 0.35770833331618923}
Saved: /content/drive/MyDrive/agentic-rag-telecom-thesis/results/planner_ragas_fixed.csv


On RAGAS (N=20), Baseline RAG achieved the highest faithfulness (0.6354) and answer relevance (0.2279), while ReAct achieved the highest context precision (0.3733). In latency (N=50), ReAct was fastest (mean 2.24s) compared to Planner–Executor (9.28s) and Baseline (11.32s).